# Emotion Vector Attribution

✏️ In this notebook, we detail our methodology for creating an 8-dimensional emotion vector for each music track. We start by assigning emotions to each unique tag in the dataset. We then derive the emotion vector for a track by aggregating the emotions associated with its tags.

This framework consists of the following steps:

1. Assign Emotions to Individual Tags
    - Generate **embeddings** using **Sentence-BERT (SBERT)** model for each tag (query) and words from the NRC Lexicon (corpus)
    - Perform **semantic search**, to retrieve the top-k closest entries from the corpus to the query by maximizing the cosine similarity on their embeddings, to identify the words that are most semantically similar to the tag
    - Perform a **weighted majority vote** to select emotions that have the highest agreement among the top-k closest words from the NRC Lexicon 


2. Derive Emotion Vector for Each Track
    - Apply **Cross-Source Normalization** to all tag occurrences
    - Select tags to ensure a *good* level of **Inter-rater Agreement** (0.75) for each track
    - Compute the **Track Emotion Vector**

In [19]:
# ~1/2min to load the model
import warnings
warnings.filterwarnings('ignore')

from utils import Emotion_Attribution
import random 
import pandas as pd
import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
sc = Emotion_Attribution()

## 1. Assign Emotions to Individual Tags

### Generate Embeddings with SBERT

We start by computing sentence embeddings for : 
- All unique tags $\rightarrow$ they are considered as queries 
- All words from NRC Lexicon $\rightarrow$ they are considered as corpus

In [21]:
# Load the data
nrc = sc.get_nrc_lexicon('data/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt') 
tags = pd.read_csv("data/tracks_tags.csv")

# Compute embeddings, ~ 30 sec. (or load them if they are already computed)
tags_embeddings, corpus_embeddings = sc.sentence_embeddings(
    tags_df=tags, nrc_df=nrc, compute_embeddings=True,
    to_save_queries="data/tags_embeddings.pt", to_save_corpus="data/corpus_embeddings.pt", 
)

### Semantic Search & Weighted Majority Vote

We retrieve for each tag the closest words in the NRC Lexicon by performing Semantic Search.


We then perform a weighted majority vote to estimate the emotion vector of each tag :

\begin{align*}
\small
&\textbf{Algorithm: Weighted Majority Vote} \\
&\textbf{Input:} \text{ Hyperparameters } \alpha_1, \alpha_2, \alpha_3, \beta; \\
&\quad \quad \text{Tag embedding } x; \\
&\quad \quad \text{Embeddings, similarity scores, and emotion vectors from the top-}k \text{ matches } \{(x_i, s_i, w_i) \mid s_u \geq s_v \text{ when } u < v\}_{i=1,...,k} \\
&\textbf{Output:} \text{ Emotion vector of the tag } v \in \mathbb{R}^8 \\
&1. \quad \text{Initialize the set of chosen matches } m : m \gets \emptyset \\
&2. \quad \text{If } s_1 \geq \alpha_1 \\
&\quad \quad \quad \text{then } m \gets \{w_1\} \\
&\quad \quad \text{else } \\
&\quad \quad \quad m \gets \{w_i \mid s_i \geq \alpha_2\} \\
&\quad \quad \quad \text{if } m = \emptyset \\
&\quad \quad \quad \quad m \gets \{w_i \mid s_i \geq \alpha_3\} \\
&3. \quad \mu \gets \sum_{w_i \in m} s_i w_i \in \mathbb{R}^8 \\
&4. \quad v \gets (v_i)_i \text{ where } v_i = 1 \text{ if } \mu_i > \beta, 0 \text{ otherwise} \\
&5. \quad \text{Return } v
\end{align*}


In [22]:
# Compute emotional vectors from words in NRC 
sc.create_emotional_vectors_nrc(nrc)

# Perform semantic search and weighted majority vote
sc.semantic_matching(tags_embeddings, corpus_embeddings, 
                     threeshold_exact = 0.95, threeshold_high = 0.9, threeshold_medium = 0.5, 
                     proportion_keep = 0.5, topk = 7)

Initial number of tags:  1580
Number of tags with emotional vector (non null):  957
Percentage of matches: 71.65 %
Percentage of no matches: 12.09 %
Percentage of exact matches: 16.27 %
Average number of matches: 3.37


In [23]:
# Save data related to tags
path_tags_to_emotions = "dataset/original/tags_to_emotions.csv"
path_tags_to_nrc_matches = "dataset/original/tags_to_nrc_matches.csv"

sc.save_tags_data(path_tags_to_emotions, path_tags_to_nrc_matches)

## Derive Emotion Vector for Each Track

### Cross-Source Normalization

Since tags come from different sources, we need to normalize their occurrences to ensure fair weighting. We divide each occurrence by the maximum occurrence encountered within the source, resulting in normalized occurrences within the [0,1] range.

In [24]:
tags = sc.normalize_occurences(tags)

### Tag Selection for Inter-rater Agreement

To reach a predefined threshold of inter-rater agreement, we perform backward selection to iteratively eliminate conflicting tags. 

Starting with the initial set of tags for a given track, we remove the tag whose removal yields the highest ICC score (*Intraclass Correlation Coefficient with one-way random effects for absolute agreement*), until the threshold is attained (0.75) or only two tags remain.

In [25]:
# ~ 10/15 min
path_tags_to_emotions = 'dataset/original/tags_to_emotions.csv'
threeshold_agreement = 0.75
min_tags_to_keep = 2

sc.select_tags_inter_rater_agreement(tags, path_tags_to_emotions, threeshold_agreement, min_tags_to_keep)

 67%|██████▋   | 3935/5892 [15:49<13:32,  2.41it/s]  

### Track Emotion Vector

We derive the emotion vector of a track by calculating the weighted average of the emotion vectors $v_i$ from the $k$ tags that demonstrated substantial inter-rater agreement, with weights set to the tags' normalized occurrences.

In [ ]:
# Track Emotion Vector (~ 15-30 seconds)
sc.create_emotion_vector_track(tags)

In [ ]:
# Save data 
path_tracks_to_emotions = "dataset/original/tracks_to_emotions.csv"
path_tracks_to_tags = "dataset/original/tracks_to_tags.csv"

sc.save_tracks_data(tags, path_tracks_to_emotions, path_tracks_to_tags)